In [1]:
import re
import json
import pickle

from sklearn.metrics import classification_report

In [290]:
with open("/Utilisateurs/umushtaq/am_reasoning/saved_models/pe_pipeline_p4_Qwen2.5-72B-Instruct-bnb-4bit/pe_pipeline_results_5.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [291]:
grounds = results['grounds']
predictions = results['predictions']

In [292]:
grounds

['### Output:\n{"argument_types": [\'C\', \'M\', \'P\', \'P\', \'P\', \'C\', \'P\', \'P\', \'P\', \'C\', \'M\']}\n{"claim_stances": [\'A\', \'S\', \'S\']}\n{"relation_types": {\'Paragraph 1\': [], \'Paragraph 2\': [(3, 6, \'S\'), (4, 6, \'S\'), (5, 6, \'S\')], \'Paragraph 3\': [(7, 10, \'S\'), (8, 9, \'S\'), (9, 10, \'S\')], \'Paragraph 4\': []}}\n',
 '### Output:\n{"argument_types": [\'M\', \'C\', \'P\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'C\', \'C\', \'M\']}\n{"claim_stances": [\'S\', \'S\', \'S\', \'A\']}\n{"relation_types": {\'Paragraph 1\': [], \'Paragraph 2\': [(3, 2, \'S\')], \'Paragraph 3\': [(5, 4, \'A\'), (6, 5, \'A\'), (7, 4, \'S\')], \'Paragraph 4\': [(8, 10, \'S\'), (9, 10, \'S\')], \'Paragraph 5\': []}}\n',
 '### Output:\n{"argument_types": [\'M\', \'C\', \'C\', \'P\', \'P\', \'P\', \'P\', \'C\', \'P\', \'P\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'M\']}\n{"claim_stances": [\'S\', \'S\', \'S\', \'S\']}\n{"relation_types": {\'Paragraph 1\': [], \

In [293]:
grounds[0].split("\n")[2]

'{"claim_stances": [\'A\', \'S\', \'S\']}'

### Process Grounds

In [294]:
def process_grounds(grounds):
    
    arg_types = []
    stance_types = []
    rel_types = []
    
    for grounds in grounds:
        
        args = grounds.split("\n")[1].replace("'", '"')
        arg_types.append(json.loads(args)["argument_types"])
        
        stances = grounds.split("\n")[2].replace("'", '"')
        #print(stances)
        stance_types.append(json.loads(stances)["claim_stances"])
        
        rels = re.sub(r'\((\d+), (\d+)\)', r'[\1, \2]', grounds.split("\n")[3])
        rels = eval(rels)["relation_types"]
        #rels = [(tuple(pair), label) for pair, label in rels]
        rel_types.append(rels)
        
    return arg_types, stance_types, rel_types

In [295]:
#grounds_acc, grounds_aric  = process_grounds(grounds)
ground_acc, ground_stances, ground_relations  = process_grounds(grounds)

In [296]:
len(ground_acc), len(ground_stances), len(ground_relations)

(80, 80, 80)

### Process Predictions

In [297]:
def process_predictions(predictions):
    
    arg_types = []
    stance_types = []
    rel_types = []
    
    for pred in predictions:
        
        args = pred.split("\n")[1].replace("'", '"')
        arg_types.append(json.loads(args)["argument_types"])
        
        stances = pred.split("\n")[2].replace("'", '"')
        #print(stances)
        stance_types.append(json.loads(stances)["claim_stances"])
        
        rels = re.sub(r'\((\d+), (\d+)\)', r'[\1, \2]', pred.split("\n")[3])
        rels = eval(rels)["relation_types"]
        #rels = [(tuple(pair), label) for pair, label in rels]
        rel_types.append(rels)
        
    return arg_types, stance_types, rel_types

In [298]:
#predictions_acc, predictions_aric  = process_predictions(predictions)
prediction_acc, prediction_stances, prediction_relations  = process_predictions(predictions)

In [299]:
len(prediction_acc), len(prediction_stances), len(prediction_relations)

(80, 80, 80)

### Compute ACC metrics

In [300]:
bad_idx = []

for idx, (g, p) in enumerate(zip(ground_acc, prediction_acc)):
    
    if len(p) != len(g):
        print(idx)
        bad_idx.append(idx)
        #predictions_acc[idx] = p[:len(g)]

In [301]:
grounds_acc = [elem for idx, elem in enumerate(ground_acc) if idx not in bad_idx]
predictions_acc = [elem for idx, elem in enumerate(prediction_acc) if idx not in bad_idx]

In [302]:
grounds = [elem for sublist in grounds_acc for elem in sublist]
predictions = [elem for sublist in predictions_acc for elem in sublist]

In [303]:
len(grounds), len(predictions)

(1266, 1266)

In [304]:
print(classification_report(grounds, predictions, digits=3))

              precision    recall  f1-score   support

           C      0.862     0.819     0.840       304
           M      0.949     0.974     0.961       153
           P      0.941     0.954     0.948       809

    accuracy                          0.924      1266
   macro avg      0.917     0.916     0.916      1266
weighted avg      0.923     0.924     0.924      1266



### Compute ARIC metrics

In [305]:
import pandas as pd

In [306]:
df = pd.read_csv("/Utilisateurs/umushtaq/am_reasoning/data_files/pe_dataset_final_c.csv", index_col=0)

In [307]:
df.columns

Index(['ann_file_name', 'argument_ids', 'argument_types', 'arg_start_bounds',
       'arg_end_bounds', 'argument_components', 'source_args', 'target_args',
       'relation_types', 'stance_types', 'essay_title', 'essay_text',
       'source_acs', 'target_acs', 'relation_pairs', 'stance_pairs', 'split',
       'sorted_acs', 'sorted_ac_types', 'para_acs', 'para_ac_types',
       'paras_structure', 'para_ac_all_pairs', 'para_ac_rels',
       'para_ac_stances'],
      dtype='object')

In [308]:
df = df[df.split == "TEST"].reset_index(drop=True)

In [309]:
df.para_ac_rels[0]

"[[(1, 2, 'NR'), (2, 1, 'NR')], [(3, 4, 'NR'), (3, 5, 'NR'), (3, 6, 'supports'), (4, 3, 'NR'), (4, 5, 'NR'), (4, 6, 'supports'), (5, 3, 'NR'), (5, 4, 'NR'), (5, 6, 'supports'), (6, 3, 'NR'), (6, 4, 'NR'), (6, 5, 'NR')], [(7, 8, 'NR'), (7, 9, 'NR'), (7, 10, 'supports'), (8, 7, 'NR'), (8, 9, 'supports'), (8, 10, 'NR'), (9, 7, 'NR'), (9, 8, 'NR'), (9, 10, 'supports'), (10, 7, 'NR'), (10, 8, 'NR'), (10, 9, 'NR')], []]"

In [310]:
def flatten(l):
    
    l = eval(l)
    return [elem for sl in l for elem in sl]

In [311]:
para_rels = []

for l in df.para_ac_rels:
    para_rels.append(flatten(l))

In [312]:
len(para_rels)

80

In [313]:
sum([len(elem) for elem in para_rels])

4922

In [314]:
ground_stances[0]

['A', 'S', 'S']

In [315]:
def get_ground_rels(l):
    
    grounds_rels = []
    
    for gr in l:
        x = list(gr.values())
        x = [elem for sl in x for elem in sl]
        grounds_rels.append(x)        
        
    return grounds_rels

In [316]:
ground_relations = get_ground_rels(ground_relations)

In [317]:
def get_preds_rels(l):
    
    preds_rels = []
    
    for pred in l:
        x = list(pred.values())
        x = [elem for sl in x for elem in sl]
        preds_rels.append(x)        
        
    return preds_rels

In [318]:
prediction_relations = get_preds_rels(prediction_relations)

In [319]:
len(ground_relations), len(prediction_relations)

(80, 80)

In [320]:
para_rels = [
    [(a, b, 'S' if relation == 'supports' else relation) for (a, b, relation) in sublist]
    for sublist in para_rels
]

In [321]:
para_rels = [
    [(a, b, 'A' if relation == 'attacks' else relation) for (a, b, relation) in sublist]
    for sublist in para_rels
]

In [322]:
len(para_rels), len(prediction_relations)

(80, 80)

In [ ]:
def get_preds(all_links_e, predictions_e):
    
    preds_p = []
    
            
    for link in all_links_e:
        
        if link in predictions_e:
            
            preds_p.append(link[2])
        else:
            preds_p.append("NR")
            
    return preds_p

In [ ]:
# def get_preds(all_links_e, predictions_e):
    
#     preds_p = []
    
            
#     for pred in predictions_e:
        
#         if pred in all_links_e:
            
#             preds_p.append(pred[2])
#         else:
#             preds_p.append("NR")
            
#     return preds_p

In [325]:
para_preds = []

for i, j in zip(para_rels, prediction_relations):
    
    para_preds.append(get_preds(i, j))

In [326]:
len(para_preds), len(para_rels)

(80, 80)

In [327]:
rels = [elem for sl in para_rels for elem in sl]
preds = [elem for sl in para_preds for elem in sl]

In [328]:
len(rels), len(preds)

(4922, 820)

In [260]:
rels = [elem[2] for elem in rels]
#predictions_rels = [elem[2] for sl in preds for elem in sl]

In [261]:
len(rels), len(preds)

(4922, 4922)

In [262]:
print(classification_report(rels, preds, digits=3))

              precision    recall  f1-score   support

           A      1.000     0.571     0.727        42
          NR      0.943     1.000     0.971      4113
           S      1.000     0.699     0.823       767

    accuracy                          0.949      4922
   macro avg      0.981     0.757     0.840      4922
weighted avg      0.952     0.949     0.945      4922



### Joint Stance + Rels

In [265]:
len(ground_stances), len(para_rels), len(para_preds), len(prediction_stances)

(80, 80, 80, 80)

In [ ]:
#### GROUNDS

In [197]:
df.columns

Index(['ann_file_name', 'argument_ids', 'argument_types', 'arg_start_bounds',
       'arg_end_bounds', 'argument_components', 'source_args', 'target_args',
       'relation_types', 'stance_types', 'essay_title', 'essay_text',
       'source_acs', 'target_acs', 'relation_pairs', 'stance_pairs', 'split',
       'sorted_acs', 'sorted_ac_types', 'para_acs', 'para_ac_types',
       'paras_structure', 'para_ac_all_pairs', 'para_ac_rels',
       'para_ac_stances'],
      dtype='object')

In [198]:
len(grounds_acc), len(predictions_acc)

(80, 80)

In [266]:
grounds_acc[0]

['C', 'M', 'P', 'P', 'P', 'C', 'P', 'P', 'P', 'C', 'M']

In [267]:
def build_stances(acc_l, stances_l):
    
    pairs = []
    
    mc_indices = [idx+1 for idx, elem in enumerate(acc_l) if elem == "M"]
    cl_indices = [idx+1 for idx, elem in enumerate(acc_l) if elem == "C"]
    
    for i, c in enumerate(cl_indices):
        label = stances_l[i]
        for mc in mc_indices:
            pairs.append((c, mc, label))
    
    return pairs

In [270]:
build_stances(grounds_acc[0], ground_stances[0])

[(1, 2, 'A'),
 (1, 11, 'A'),
 (6, 2, 'S'),
 (6, 11, 'S'),
 (10, 2, 'S'),
 (10, 11, 'S')]

In [272]:
ground_acc[0], ground_stances[0]

(['C', 'M', 'P', 'P', 'P', 'C', 'P', 'P', 'P', 'C', 'M'], ['A', 'S', 'S'])

In [273]:
ground_stances_pairs = []

for i, j in zip(ground_acc, ground_stances):
    ground_stances_pairs.append(build_stances(i, j))

In [274]:
ground_stances_pairs[0]

[(1, 2, 'A'),
 (1, 11, 'A'),
 (6, 2, 'S'),
 (6, 11, 'S'),
 (10, 2, 'S'),
 (10, 11, 'S')]

In [280]:
para_rels[0]

[(1, 2, 'NR'),
 (2, 1, 'NR'),
 (3, 4, 'NR'),
 (3, 5, 'NR'),
 (3, 6, 'S'),
 (4, 3, 'NR'),
 (4, 5, 'NR'),
 (4, 6, 'S'),
 (5, 3, 'NR'),
 (5, 4, 'NR'),
 (5, 6, 'S'),
 (6, 3, 'NR'),
 (6, 4, 'NR'),
 (6, 5, 'NR'),
 (7, 8, 'NR'),
 (7, 9, 'NR'),
 (7, 10, 'S'),
 (8, 7, 'NR'),
 (8, 9, 'S'),
 (8, 10, 'NR'),
 (9, 7, 'NR'),
 (9, 8, 'NR'),
 (9, 10, 'S'),
 (10, 7, 'NR'),
 (10, 8, 'NR'),
 (10, 9, 'NR')]

In [281]:
def add_grounds(ground_stances_pairs, para_rels, para_preds):
    
    for idx, (ground_stances_pair, para_rel, para_pred) in enumerate(zip(ground_stances_pairs, para_rels, para_preds)):
        
        for i, j, r in ground_stances_pair:
            
            if (i, j, "NR") in para_rel:
                
                index = para_rel.index((i, j, "NR"))
                para_rel[index] = (i, j, r)
                
            else:
            
                para_rel.append((i, j, r))
                
            if (i, j, "NR") in para_pred or (i, j, "A") in para_pred or (i, j, "S") in para_pred:
                
                pass
            else:
            
                para_pred.append((i, j, "NR"))
                
    return para_rels, para_preds

In [284]:
for i, j in zip(para_rels, para_preds):
    print(len(i), len(j))

31 32
29 30
91 92
91 92
36 38
66 67
101 102
39 40
16 16
49 50
44 44
32 33
49 50
39 40
126 126
26 26
16 17
60 60
44 45
71 71
37 38
120 120
52 54
43 44
38 38
58 60
164 165
149 152
50 50
44 44
160 162
128 128
112 112
79 82
52 52
20 20
57 58
42 44
129 129
81 82
53 54
114 114
82 82
89 90
98 102
47 48
82 84
79 80
30 30
34 34
123 124
48 48
109 112
54 54
47 48
93 94
74 76
39 40
71 72
52 56
123 127
60 62
50 52
104 106
66 66
62 66
91 94
87 88
14 14
90 90
100 100
39 40
58 58
78 80
71 72
105 106
25 26
25 26
27 28
60 60


In [283]:
para_rels, para_preds = add_grounds(ground_stances_pairs, para_rels, para_preds)

In [275]:
pred_stances_pairs = []

for idx, (i, j) in enumerate(zip(prediction_acc, prediction_stances)):
    try:
        pred_stances_pairs.append(build_stances(i, j))
    except:
        print(idx)
        pred_stances_pairs.append([])

27
30
33
52


In [276]:
pred_stances_pairs[0]

[(1, 2, 'A'),
 (1, 11, 'A'),
 (6, 2, 'S'),
 (6, 11, 'S'),
 (10, 2, 'S'),
 (10, 11, 'S')]

In [278]:
para_rels[0]

[(1, 2, 'NR'),
 (2, 1, 'NR'),
 (3, 4, 'NR'),
 (3, 5, 'NR'),
 (3, 6, 'S'),
 (4, 3, 'NR'),
 (4, 5, 'NR'),
 (4, 6, 'S'),
 (5, 3, 'NR'),
 (5, 4, 'NR'),
 (5, 6, 'S'),
 (6, 3, 'NR'),
 (6, 4, 'NR'),
 (6, 5, 'NR'),
 (7, 8, 'NR'),
 (7, 9, 'NR'),
 (7, 10, 'S'),
 (8, 7, 'NR'),
 (8, 9, 'S'),
 (8, 10, 'NR'),
 (9, 7, 'NR'),
 (9, 8, 'NR'),
 (9, 10, 'S'),
 (10, 7, 'NR'),
 (10, 8, 'NR'),
 (10, 9, 'NR')]

In [ ]:
def get_preds(all_links_e, predictions_e):
    
    preds_p = []
            
    for link in all_links_e:
        
        if link in predictions_e:
            
            preds_p.append(link[2])
        else:
            preds_p.append("NR")
            
    return preds_p

In [239]:
preds = []

for i, j in zip(para_rels, predictions_l):
    
    preds.append(get_preds(i, j))

In [240]:
len(preds)

80

In [ ]:
grounds = []

for i, j in zip(para_rels, grounds_l):
    
    grounds.append(get_preds(i, j))

In [218]:
ground_stances[27]

['S', 'A', 'S', 'S', 'S', 'S']

In [219]:
predictions_stances[27]

['S', 'S', 'S', 'S', 'A']

In [220]:
predictions_acc[27]

['M',
 'C',
 'C',
 'P',
 'P',
 'P',
 'P',
 'C',
 'P',
 'P',
 'P',
 'P',
 'P',
 'P',
 'P',
 'C',
 'P',
 'P',
 'P',
 'P',
 'P',
 'P',
 'P',
 'C',
 'M',
 'C']

In [154]:
grounds_rels = [v for gr in ground_relations for k,v in gr.items()]

In [117]:
para_stances = []

for l in df.para_ac_stances:
    para_stances.append(flatten(l))

In [118]:
sum([len(elem) for elem in para_stances])/3

586.0

In [119]:
df.sorted_ac_types

0     ['Claim', 'MajorClaim', 'Premise', 'Premise', ...
1     ['MajorClaim', 'Claim', 'Premise', 'Claim', 'P...
2     ['MajorClaim', 'Claim', 'Claim', 'Premise', 'P...
3     ['Claim', 'Premise', 'Premise', 'MajorClaim', ...
4     ['Claim', 'Premise', 'Premise', 'Premise', 'Pr...
                            ...                        
75    ['MajorClaim', 'Premise', 'Premise', 'Premise'...
76    ['MajorClaim', 'Claim', 'Premise', 'Premise', ...
77    ['MajorClaim', 'Claim', 'Premise', 'Premise', ...
78    ['MajorClaim', 'Premise', 'Premise', 'Claim', ...
79    ['Claim', 'Claim', 'Premise', 'Premise', 'Prem...
Name: sorted_ac_types, Length: 80, dtype: object

In [120]:
acs = [eval(elem) for elem in df.sorted_ac_types]

In [121]:
len(acs)

80

In [122]:
x = [elem for sl in acs for elem in sl]

In [123]:
len(x)

1266

In [124]:
x.count('MajorClaim'), x.count('Claim'), x.count('Premise')

(153, 304, 809)

In [125]:
all_links = []

for i, j in zip(para_rels, df.para_ac_stances):
    j = eval(j)
    all_links.append(i + j)

In [126]:
len(all_links)

80

In [127]:
sum([len(elem) for elem in all_links])

5508

In [131]:
all_links

[[(1, 2, 'NR'),
  (2, 1, 'NR'),
  (3, 4, 'NR'),
  (3, 5, 'NR'),
  (3, 6, 'S'),
  (4, 3, 'NR'),
  (4, 5, 'NR'),
  (4, 6, 'S'),
  (5, 3, 'NR'),
  (5, 4, 'NR'),
  (5, 6, 'S'),
  (6, 3, 'NR'),
  (6, 4, 'NR'),
  (6, 5, 'NR'),
  (7, 8, 'NR'),
  (7, 9, 'NR'),
  (7, 10, 'S'),
  (8, 7, 'NR'),
  (8, 9, 'S'),
  (8, 10, 'NR'),
  (9, 7, 'NR'),
  (9, 8, 'NR'),
  (9, 10, 'S'),
  (10, 7, 'NR'),
  (10, 8, 'NR'),
  (10, 9, 'NR'),
  (1, 2, 'attacks'),
  (1, 11, 'attacks'),
  (6, 2, 'S'),
  (6, 11, 'S'),
  (10, 2, 'S'),
  (10, 11, 'S')],
 [(2, 3, 'NR'),
  (3, 2, 'S'),
  (4, 5, 'NR'),
  (4, 6, 'NR'),
  (4, 7, 'NR'),
  (5, 4, 'attacks'),
  (5, 6, 'NR'),
  (5, 7, 'NR'),
  (6, 4, 'NR'),
  (6, 5, 'attacks'),
  (6, 7, 'NR'),
  (7, 4, 'S'),
  (7, 5, 'NR'),
  (7, 6, 'NR'),
  (8, 9, 'NR'),
  (8, 10, 'S'),
  (9, 8, 'NR'),
  (9, 10, 'S'),
  (10, 8, 'NR'),
  (10, 9, 'NR'),
  (11, 12, 'NR'),
  (12, 11, 'NR'),
  (2, 1, 'S'),
  (2, 12, 'S'),
  (4, 1, 'S'),
  (4, 12, 'S'),
  (10, 1, 'S'),
  (10, 12, 'S'),
  (11, 1, 'atta

In [130]:
all_links = [
    [(a, b, 'S' if relation == 'supports' else relation) for (a, b, relation) in sublist]
    for sublist in all_links
]

In [132]:
all_links = [
    [(a, b, 'A' if relation == 'attacks' else relation) for (a, b, relation) in sublist]
    for sublist in all_links
]

In [133]:
predictions_aric[0]

[(3, 6, 'S'),
 (4, 6, 'S'),
 (5, 6, 'S'),
 (7, 10, 'S'),
 (8, 10, 'S'),
 (9, 10, 'S'),
 (1, 2, 'A'),
 (1, 11, 'A'),
 (6, 2, 'S'),
 (6, 11, 'S'),
 (10, 2, 'S'),
 (10, 11, 'S')]

In [134]:
def get_preds(all_links_e, predictions_e):
    
    preds_p = []
    
    # for pred in predictions_e:
        
    #     if pred in all_links_e:
            
    #         correct_predictions.append(pred[2])
            
    for link in all_links_e:
        
        if link in predictions_e:
            
            preds_p.append(link[2])
        else:
            preds_p.append("NR")
            
    return preds_p

In [ ]:
preds = []

for i, j in zip(all_links, predictions_aric):
    
    preds.append(get_preds(i, j))

In [136]:
len(preds)

80

In [137]:
preds = [elem for sl in preds for elem in sl]

In [138]:
len(preds)

5508

In [139]:
preds.count("NR")

4507

In [141]:
all_links

[[(1, 2, 'NR'),
  (2, 1, 'NR'),
  (3, 4, 'NR'),
  (3, 5, 'NR'),
  (3, 6, 'S'),
  (4, 3, 'NR'),
  (4, 5, 'NR'),
  (4, 6, 'S'),
  (5, 3, 'NR'),
  (5, 4, 'NR'),
  (5, 6, 'S'),
  (6, 3, 'NR'),
  (6, 4, 'NR'),
  (6, 5, 'NR'),
  (7, 8, 'NR'),
  (7, 9, 'NR'),
  (7, 10, 'S'),
  (8, 7, 'NR'),
  (8, 9, 'S'),
  (8, 10, 'NR'),
  (9, 7, 'NR'),
  (9, 8, 'NR'),
  (9, 10, 'S'),
  (10, 7, 'NR'),
  (10, 8, 'NR'),
  (10, 9, 'NR'),
  (1, 2, 'A'),
  (1, 11, 'A'),
  (6, 2, 'S'),
  (6, 11, 'S'),
  (10, 2, 'S'),
  (10, 11, 'S')],
 [(2, 3, 'NR'),
  (3, 2, 'S'),
  (4, 5, 'NR'),
  (4, 6, 'NR'),
  (4, 7, 'NR'),
  (5, 4, 'A'),
  (5, 6, 'NR'),
  (5, 7, 'NR'),
  (6, 4, 'NR'),
  (6, 5, 'A'),
  (6, 7, 'NR'),
  (7, 4, 'S'),
  (7, 5, 'NR'),
  (7, 6, 'NR'),
  (8, 9, 'NR'),
  (8, 10, 'S'),
  (9, 8, 'NR'),
  (9, 10, 'S'),
  (10, 8, 'NR'),
  (10, 9, 'NR'),
  (11, 12, 'NR'),
  (12, 11, 'NR'),
  (2, 1, 'S'),
  (2, 12, 'S'),
  (4, 1, 'S'),
  (4, 12, 'S'),
  (10, 1, 'S'),
  (10, 12, 'S'),
  (11, 1, 'A'),
  (11, 12, 'A')],
 [(1,

In [142]:
all_links = [elem[2] for sl in all_links for elem in sl]

In [144]:
len(all_links), len(preds)

(5508, 5508)

In [145]:
print(classification_report(all_links, preds, digits=3))

              precision    recall  f1-score   support

           A      1.000     0.512     0.677       129
          NR      0.913     1.000     0.954      4113
           S      1.000     0.739     0.850      1266

    accuracy                          0.928      5508
   macro avg      0.971     0.750     0.827      5508
weighted avg      0.935     0.928     0.924      5508



In [31]:
len(grounds_aric), len(predictions_aric)

(80, 80)

In [18]:
grounds_aric[0]

[(8, 4, 'S'),
 (7, 4, 'S'),
 (6, 4, 'S'),
 (10, 11, 'S'),
 (9, 5, 'S'),
 (11, 5, 'S'),
 (3, 1, 'A'),
 (3, 2, 'A'),
 (4, 1, 'S'),
 (4, 2, 'S'),
 (5, 1, 'S'),
 (5, 2, 'S')]

In [19]:
predictions_aric[0]

[(6, 4, 'S'),
 (7, 4, 'S'),
 (8, 4, 'S'),
 (9, 5, 'S'),
 (10, 5, 'S'),
 (11, 5, 'S'),
 (3, 1, 'A'),
 (3, 2, 'A'),
 (4, 1, 'S'),
 (4, 2, 'S'),
 (5, 1, 'S'),
 (5, 2, 'S')]

In [20]:
nr_acs = [len(elem) for elem in grounds_acc]

In [21]:
def build_grounds(grounds, n_acs):
    
    ground_pairs = [(elem[0], elem[1]) for elem in grounds]
    all_triples = [(i, j, "NR") for i in range(1, n_acs+1) for j in range(1, n_acs+1) if (i != j and (i, j) not in ground_pairs)]
    
    #grounds_t = [(elem[0][0], elem[0][1], elem[1]) for elem in grounds]
    
    grounds_t = [(elem[0], elem[1], elem[2]) for elem in grounds]
    
    return all_triples + grounds_t

In [22]:
grounds_triples = []

for ground, n_acs in zip(grounds_aric, nr_acs):
    
    grounds_triples.append(build_grounds(ground, n_acs))

In [23]:
len(grounds_triples)

80

In [24]:
grounds_triples[0]

[(1, 2, 'NR'),
 (1, 3, 'NR'),
 (1, 4, 'NR'),
 (1, 5, 'NR'),
 (1, 6, 'NR'),
 (1, 7, 'NR'),
 (1, 8, 'NR'),
 (1, 9, 'NR'),
 (1, 10, 'NR'),
 (1, 11, 'NR'),
 (2, 1, 'NR'),
 (2, 3, 'NR'),
 (2, 4, 'NR'),
 (2, 5, 'NR'),
 (2, 6, 'NR'),
 (2, 7, 'NR'),
 (2, 8, 'NR'),
 (2, 9, 'NR'),
 (2, 10, 'NR'),
 (2, 11, 'NR'),
 (3, 4, 'NR'),
 (3, 5, 'NR'),
 (3, 6, 'NR'),
 (3, 7, 'NR'),
 (3, 8, 'NR'),
 (3, 9, 'NR'),
 (3, 10, 'NR'),
 (3, 11, 'NR'),
 (4, 3, 'NR'),
 (4, 5, 'NR'),
 (4, 6, 'NR'),
 (4, 7, 'NR'),
 (4, 8, 'NR'),
 (4, 9, 'NR'),
 (4, 10, 'NR'),
 (4, 11, 'NR'),
 (5, 3, 'NR'),
 (5, 4, 'NR'),
 (5, 6, 'NR'),
 (5, 7, 'NR'),
 (5, 8, 'NR'),
 (5, 9, 'NR'),
 (5, 10, 'NR'),
 (5, 11, 'NR'),
 (6, 1, 'NR'),
 (6, 2, 'NR'),
 (6, 3, 'NR'),
 (6, 5, 'NR'),
 (6, 7, 'NR'),
 (6, 8, 'NR'),
 (6, 9, 'NR'),
 (6, 10, 'NR'),
 (6, 11, 'NR'),
 (7, 1, 'NR'),
 (7, 2, 'NR'),
 (7, 3, 'NR'),
 (7, 5, 'NR'),
 (7, 6, 'NR'),
 (7, 8, 'NR'),
 (7, 9, 'NR'),
 (7, 10, 'NR'),
 (7, 11, 'NR'),
 (8, 1, 'NR'),
 (8, 2, 'NR'),
 (8, 3, 'NR'),
 (8, 5, 'NR

In [25]:
def build_predictions(ground, pred, n_acs):
    
    predictions = []
    
    for p in pred:
        
        if p in ground:
            
            predictions.append(p)
    
    prediction_pairs = [(elem[0], elem[1]) for elem in predictions]
    #all_triples = [(i, j, "NR") for i in range(1, n_acs+1) for j in range(1, n_acs+1) if (i != j or (i, j) not in prediction_pairs)]
    all_triples = [(i, j, "NR") for i in range(1, n_acs+1) for j in range(1, n_acs+1) if (i != j and (i, j) not in prediction_pairs)]

    
    #preds_t = [(elem[0][0], elem[0][1], elem[1]) for elem in predictions]
    preds_t = [(elem[0], elem[1], elem[2]) for elem in predictions]
    #print(preds_t)
    
    return all_triples + preds_t

In [26]:
prediction_triples = []

for ground, pred, n_acs in zip(grounds_aric, predictions_aric, nr_acs):
    
    prediction_triples.append(build_predictions(ground, pred, n_acs))
    #break

In [27]:
prediction_triples[0]

[(1, 2, 'NR'),
 (1, 3, 'NR'),
 (1, 4, 'NR'),
 (1, 5, 'NR'),
 (1, 6, 'NR'),
 (1, 7, 'NR'),
 (1, 8, 'NR'),
 (1, 9, 'NR'),
 (1, 10, 'NR'),
 (1, 11, 'NR'),
 (2, 1, 'NR'),
 (2, 3, 'NR'),
 (2, 4, 'NR'),
 (2, 5, 'NR'),
 (2, 6, 'NR'),
 (2, 7, 'NR'),
 (2, 8, 'NR'),
 (2, 9, 'NR'),
 (2, 10, 'NR'),
 (2, 11, 'NR'),
 (3, 4, 'NR'),
 (3, 5, 'NR'),
 (3, 6, 'NR'),
 (3, 7, 'NR'),
 (3, 8, 'NR'),
 (3, 9, 'NR'),
 (3, 10, 'NR'),
 (3, 11, 'NR'),
 (4, 3, 'NR'),
 (4, 5, 'NR'),
 (4, 6, 'NR'),
 (4, 7, 'NR'),
 (4, 8, 'NR'),
 (4, 9, 'NR'),
 (4, 10, 'NR'),
 (4, 11, 'NR'),
 (5, 3, 'NR'),
 (5, 4, 'NR'),
 (5, 6, 'NR'),
 (5, 7, 'NR'),
 (5, 8, 'NR'),
 (5, 9, 'NR'),
 (5, 10, 'NR'),
 (5, 11, 'NR'),
 (6, 1, 'NR'),
 (6, 2, 'NR'),
 (6, 3, 'NR'),
 (6, 5, 'NR'),
 (6, 7, 'NR'),
 (6, 8, 'NR'),
 (6, 9, 'NR'),
 (6, 10, 'NR'),
 (6, 11, 'NR'),
 (7, 1, 'NR'),
 (7, 2, 'NR'),
 (7, 3, 'NR'),
 (7, 5, 'NR'),
 (7, 6, 'NR'),
 (7, 8, 'NR'),
 (7, 9, 'NR'),
 (7, 10, 'NR'),
 (7, 11, 'NR'),
 (8, 1, 'NR'),
 (8, 2, 'NR'),
 (8, 3, 'NR'),
 (8, 5, 'NR

In [28]:
bad_idx = []

for idx, (g, p) in enumerate(zip(grounds_triples, prediction_triples)):
    
    if len(p) != len(g):
        print(idx, len(g), len(p))
        bad_idx.append(idx)

In [29]:
nr_acs[70]

16

In [30]:
grounds_triples[70]

[(1, 2, 'NR'),
 (1, 3, 'NR'),
 (1, 4, 'NR'),
 (1, 5, 'NR'),
 (1, 6, 'NR'),
 (1, 7, 'NR'),
 (1, 8, 'NR'),
 (1, 9, 'NR'),
 (1, 10, 'NR'),
 (1, 11, 'NR'),
 (1, 12, 'NR'),
 (1, 13, 'NR'),
 (1, 14, 'NR'),
 (1, 15, 'NR'),
 (1, 16, 'NR'),
 (2, 1, 'NR'),
 (2, 3, 'NR'),
 (2, 4, 'NR'),
 (2, 5, 'NR'),
 (2, 6, 'NR'),
 (2, 7, 'NR'),
 (2, 9, 'NR'),
 (2, 10, 'NR'),
 (2, 11, 'NR'),
 (2, 12, 'NR'),
 (2, 13, 'NR'),
 (2, 14, 'NR'),
 (2, 15, 'NR'),
 (2, 16, 'NR'),
 (3, 1, 'NR'),
 (3, 2, 'NR'),
 (3, 5, 'NR'),
 (3, 6, 'NR'),
 (3, 7, 'NR'),
 (3, 8, 'NR'),
 (3, 9, 'NR'),
 (3, 10, 'NR'),
 (3, 11, 'NR'),
 (3, 12, 'NR'),
 (3, 13, 'NR'),
 (3, 14, 'NR'),
 (3, 15, 'NR'),
 (3, 16, 'NR'),
 (4, 1, 'NR'),
 (4, 2, 'NR'),
 (4, 3, 'NR'),
 (4, 5, 'NR'),
 (4, 6, 'NR'),
 (4, 7, 'NR'),
 (4, 9, 'NR'),
 (4, 10, 'NR'),
 (4, 11, 'NR'),
 (4, 12, 'NR'),
 (4, 13, 'NR'),
 (4, 14, 'NR'),
 (4, 15, 'NR'),
 (4, 16, 'NR'),
 (5, 1, 'NR'),
 (5, 2, 'NR'),
 (5, 3, 'NR'),
 (5, 4, 'NR'),
 (5, 6, 'NR'),
 (5, 7, 'NR'),
 (5, 9, 'NR'),
 (5, 10, 'NR

In [31]:
prediction_triples[70]

[(1, 2, 'NR'),
 (1, 3, 'NR'),
 (1, 4, 'NR'),
 (1, 5, 'NR'),
 (1, 6, 'NR'),
 (1, 7, 'NR'),
 (1, 8, 'NR'),
 (1, 9, 'NR'),
 (1, 10, 'NR'),
 (1, 11, 'NR'),
 (1, 12, 'NR'),
 (1, 13, 'NR'),
 (1, 14, 'NR'),
 (1, 15, 'NR'),
 (1, 16, 'NR'),
 (2, 1, 'NR'),
 (2, 3, 'NR'),
 (2, 4, 'NR'),
 (2, 5, 'NR'),
 (2, 6, 'NR'),
 (2, 7, 'NR'),
 (2, 9, 'NR'),
 (2, 10, 'NR'),
 (2, 11, 'NR'),
 (2, 12, 'NR'),
 (2, 13, 'NR'),
 (2, 14, 'NR'),
 (2, 15, 'NR'),
 (2, 16, 'NR'),
 (3, 1, 'NR'),
 (3, 2, 'NR'),
 (3, 4, 'NR'),
 (3, 5, 'NR'),
 (3, 6, 'NR'),
 (3, 7, 'NR'),
 (3, 8, 'NR'),
 (3, 9, 'NR'),
 (3, 10, 'NR'),
 (3, 11, 'NR'),
 (3, 12, 'NR'),
 (3, 13, 'NR'),
 (3, 14, 'NR'),
 (3, 15, 'NR'),
 (3, 16, 'NR'),
 (4, 1, 'NR'),
 (4, 2, 'NR'),
 (4, 3, 'NR'),
 (4, 5, 'NR'),
 (4, 6, 'NR'),
 (4, 7, 'NR'),
 (4, 9, 'NR'),
 (4, 10, 'NR'),
 (4, 11, 'NR'),
 (4, 12, 'NR'),
 (4, 13, 'NR'),
 (4, 14, 'NR'),
 (4, 15, 'NR'),
 (4, 16, 'NR'),
 (5, 1, 'NR'),
 (5, 2, 'NR'),
 (5, 3, 'NR'),
 (5, 4, 'NR'),
 (5, 6, 'NR'),
 (5, 7, 'NR'),
 (5, 9, 'NR'

In [32]:
bad_idx

[]

In [33]:
grounds_triples = [val for i, val in enumerate(grounds_triples) if i not in bad_idx]
prediction_triples = [val for i, val in enumerate(prediction_triples) if i not in bad_idx]

In [34]:
grounds_l = [elem for sublist in grounds_triples for elem in sublist]
predictions_l = [elem for sublist in prediction_triples for elem in sublist]

In [35]:
len(grounds_l), len(predictions_l)

(20192, 20192)

In [36]:
grounds = [elem[2] for elem in grounds_l]
predictions = [elem[2] for elem in predictions_l]

In [37]:
print(classification_report(grounds, predictions, digits=3))

              precision    recall  f1-score   support

           A      0.705     0.481     0.571       129
          NR      0.978     1.000     0.989     18797
           S      0.951     0.668     0.785      1266

    accuracy                          0.976     20192
   macro avg      0.878     0.716     0.782     20192
weighted avg      0.975     0.976     0.974     20192



### Compute ARC metrics

In [90]:
3820 + 1021 + 405 + 92

5338

In [91]:
1021 + 92

1113

In [92]:
3832 + 1506

5338

In [ ]:
grounds_ari

[['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'A', 'A', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'A', 'S', 'S'],
 ['S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A', 'A', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'A', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S

In [106]:
grounds_ari

[[(8, 4), (7, 4), (6, 4), (10, 11), (9, 5), (11, 5)],
 [(6, 3), (7, 4), (8, 7), (9, 4), (10, 5), (11, 5)],
 [(7, 4),
  (8, 9),
  (10, 9),
  (9, 4),
  (11, 5),
  (12, 5),
  (13, 6),
  (14, 6),
  (15, 6),
  (16, 6),
  (17, 16),
  (18, 16),
  (19, 16)],
 [(5, 3),
  (4, 3),
  (9, 6),
  (11, 13),
  (12, 13),
  (10, 6),
  (13, 6),
  (14, 7),
  (15, 7),
  (18, 8),
  (17, 16),
  (16, 8)],
 [(6, 4), (7, 4), (8, 7), (9, 7), (10, 5), (11, 5)],
 [(5, 2),
  (6, 2),
  (7, 2),
  (8, 2),
  (9, 2),
  (10, 2),
  (11, 4),
  (4, 3),
  (12, 13),
  (13, 3)],
 [(11, 10),
  (12, 10),
  (7, 4),
  (8, 4),
  (9, 4),
  (10, 4),
  (13, 5),
  (14, 5),
  (15, 5),
  (16, 5),
  (17, 6),
  (18, 6),
  (19, 6),
  (20, 6),
  (21, 6)],
 [(6, 4), (8, 9), (7, 9), (9, 5), (10, 5), (11, 5)],
 [(6, 3), (7, 4), (8, 4), (9, 5)],
 [(5, 4), (6, 4), (7, 4), (8, 4), (11, 9), (12, 13), (9, 13)],
 [(8, 3),
  (9, 3),
  (10, 3),
  (11, 3),
  (12, 13),
  (13, 4),
  (14, 4),
  (6, 5),
  (7, 5)],
 [(4, 3), (6, 7), (7, 5), (9, 10)],
 [(9, 3)

In [107]:
aric_grounds = []
    
for rel_pairs, rel_types in zip(grounds_ari, grounds_arc):
    
    result = list(zip(rel_pairs, rel_types))
    aric_grounds.append(result)

In [108]:
len(aric_grounds)

80

In [109]:
aric_grounds[0]

[((8, 4), 'S'),
 ((7, 4), 'S'),
 ((6, 4), 'S'),
 ((10, 11), 'S'),
 ((9, 5), 'S'),
 ((11, 5), 'S')]

In [110]:
aric_predictions = []
    
for rel_pairs, rel_types in zip(predictions_ari, predictions_arc):
        
    result = list(zip(rel_pairs, rel_types))
    aric_predictions.append(result)

In [111]:
len(aric_predictions)

80

In [112]:
aric_predictions[0]

[((7, 4), 'S'), ((8, 4), 'S'), ((10, 5), 'S'), ((11, 5), 'S'), ((6, 4), 'S')]

In [172]:
len(nr_acs)

80

In [175]:
len(aric_grounds), len(aric_predictions)

(80, 80)

In [176]:
aric_grounds[0]

[((8, 4), 'S'),
 ((7, 4), 'S'),
 ((6, 4), 'S'),
 ((10, 11), 'S'),
 ((9, 5), 'S'),
 ((11, 5), 'S')]

In [177]:
aric_predictions[0]

[((7, 4), 'S'), ((8, 4), 'S'), ((10, 5), 'S'), ((11, 5), 'S'), ((6, 4), 'S')]

In [178]:
g_0 = set(aric_grounds[0])

In [179]:
g_0

{((6, 4), 'S'),
 ((7, 4), 'S'),
 ((8, 4), 'S'),
 ((9, 5), 'S'),
 ((10, 11), 'S'),
 ((11, 5), 'S')}

In [180]:
p_0 = set(aric_predictions[0])

In [181]:
p_0

{((6, 4), 'S'), ((7, 4), 'S'), ((8, 4), 'S'), ((10, 5), 'S'), ((11, 5), 'S')}

In [182]:
z = g_0.intersection(p_0)

In [183]:
z

{((6, 4), 'S'), ((7, 4), 'S'), ((8, 4), 'S'), ((11, 5), 'S')}

In [121]:
correct_predictions = []
correct_grounds = []

for aric_ground, aric_pred in zip(aric_grounds, aric_predictions):
    
    common_elements = [item for item in aric_pred if item in aric_ground]
            
    correct_predictions.append(common_elements)
    correct_grounds.append(aric_ground)

In [122]:
len(correct_predictions), len(correct_grounds)

(80, 80)

In [123]:
correct_predictions[0]

[((7, 4), 'S'), ((8, 4), 'S'), ((11, 5), 'S'), ((6, 4), 'S')]

In [124]:
correct_grounds[0]

[((8, 4), 'S'),
 ((7, 4), 'S'),
 ((6, 4), 'S'),
 ((10, 11), 'S'),
 ((9, 5), 'S'),
 ((11, 5), 'S')]

In [128]:
ground_types = [[item[1] for item in sublist] for sublist in correct_grounds]

In [130]:
len(ground_types)

80

In [131]:
prediction_types = [[item[1] for item in sublist] for sublist in correct_predictions]

In [132]:
len(prediction_types)

80

In [173]:
ground_types

[['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'A', 'A', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'A', 'S', 'S'],
 ['S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A', 'A', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'A', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S

In [174]:
prediction_types

[['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S'],
 ['S', 'S', 'S'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S'],
 ['S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S'],
 ['S', 'S'],
 ['S', 'S', 'S', 'S', 'A'],
 ['S', 'S'],
 ['S', 'S', 'S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S'],
 ['S', 'S', 'S'],
 ['S', 'S', 'A'],
 ['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S'],
 ['S', 'S'],
 ['S'],
 ['S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S',
  'S'],
 ['S', 'S'],
 ['S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'A', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'A'],
 [],
 ['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'S', 'S

In [187]:
bad_idx = []
oppo_d = {"S": "A", "A": "S"}

for idx, (i,j) in enumerate(zip(ground_types, prediction_types)):
    
    if len(i) > len(j):
        prediction_types[idx] = prediction_types[idx] + [oppo_d[elem] for elem in ground_types[idx][len(j):]]
        
    elif len(j) > len(i):        
        prediction_types[idx] = prediction_types[idx][:len(i)]
        #print(idx, len(i), len(j))
        #bad_idx.append(idx)

In [188]:
grounds = [elem for sublist in ground_types for elem in sublist]
predictions = [elem for sublist in prediction_types for elem in sublist]

In [189]:
len(grounds), len(predictions)

(809, 809)

In [190]:
print(classification_report(grounds, predictions, digits=3))

              precision    recall  f1-score   support

           A      0.005     0.048     0.009        42
           S      0.904     0.493     0.638       767

    accuracy                          0.470       809
   macro avg      0.455     0.270     0.324       809
weighted avg      0.858     0.470     0.605       809



In [ ]:
### Maybe it makes no sense to do ari and arc separately.

In [ ]:
{"relation_types": {"Paragraph1" : [],
                "Paragraph2" : [(4, 3, 'S'), (5, 3, 'S'), (6, 3, 'S')],
                "Paragraph_3" : [(8, 7, 'S')],
                "Paragraph_4" : [(10, 11, 'S'), (9, 11, 'S')]}}